# Analyze data from education systems

In [1]:
import pandas as pd
from helpers import df_initial_analysis

## Initial data analysis

In [2]:
country = pd.read_csv("datasets/EdStatsCountry.csv")
df_initial_analysis(country, "country")
df_initial_analysis

fasf

The country dataset shape is: 241 rows & 32 columns
Total of NaN values in the country dataset is: 2354
Percentage of NaN values in the country dataset is: 30.52 %
There are 0 full duplicates rows
There are 1 empty columns
   The empty columns are: ['Unnamed: 31']
------------------------------------------------------------------------

Number of values/records per columns in country dataset
Country Code                                         241
Long Name                                            241
Short Name                                           241
Table Name                                           241
WB-2 code                                            240
2-alpha code                                         238
Currency Unit                                        215
System of National Accounts                          215
Region                                               214
Income Group                                         214
Latest population census     

<function helpers.df_initial_analysis(df, name_df)>

In [3]:
serie = pd.read_csv("datasets/EdStatsSeries.csv")
df_initial_analysis(serie, "serie")

fasf

The serie dataset shape is: 3665 rows & 21 columns
Total of NaN values in the serie dataset is: 55203
Percentage of NaN values in the serie dataset is: 71.72 %
There are 0 full duplicates rows
There are 6 empty columns
   The empty columns are: ['Unit of measure', 'Notes from original source', 'Other web links', 'Related indicators', 'License Type', 'Unnamed: 20']
------------------------------------------------------------------------

Number of values/records per columns in serie dataset
Series Code                            3665
Source                                 3665
Indicator Name                         3665
Long definition                        3665
Topic                                  3665
Short definition                       2156
Other notes                             552
Base Period                             314
Related source links                    215
Periodicity                              99
Aggregation method                       47
Statistical con

In [4]:
country_serie = pd.read_csv("datasets/EdStatsCountry-Series.csv")
df_initial_analysis(country_serie, "country_serie")

fasf

The country_serie dataset shape is: 613 rows & 4 columns
Total of NaN values in the country_serie dataset is: 613
Percentage of NaN values in the country_serie dataset is: 25.0 %
There are 0 full duplicates rows
There are 1 empty columns
   The empty columns are: ['Unnamed: 3']
------------------------------------------------------------------------

Number of values/records per columns in country_serie dataset
CountryCode    613
SeriesCode     613
DESCRIPTION    613
Unnamed: 3       0
dtype: int64


In [5]:
df_foot_note = pd.read_csv("datasets/EdStatsFootNote.csv")
df_initial_analysis(df_foot_note, "Foot Note")

fasf

The Foot Note dataset shape is: 643638 rows & 5 columns
Total of NaN values in the Foot Note dataset is: 643638
Percentage of NaN values in the Foot Note dataset is: 20.0 %
There are 0 full duplicates rows
There are 1 empty columns
   The empty columns are: ['Unnamed: 4']
------------------------------------------------------------------------

Number of values/records per columns in Foot Note dataset
CountryCode    643638
SeriesCode     643638
Year           643638
DESCRIPTION    643638
Unnamed: 4          0
dtype: int64


In [6]:
data = pd.read_csv("datasets/EdStatsData.csv")
df_initial_analysis(data, "Data")

fasf



## Data cleanup

In [7]:
# List of countries as given in ISO 3166-1 and the corresponding ISO 3166-1-alpha-2 code elements.  
countries_iso = pd.read_csv("datasets/countries_ISO.csv", sep=",", keep_default_na=False, 
                            encoding='utf-8') # Avoiding that "NA" will be interpreted as NaN by Pandas
df_initial_analysis(countries_iso, "Countries ISO")

The Countries ISO dataset shape is: 249 rows & 2 columns
Total of NaN values in the Countries ISO dataset is: 0
Percentage of NaN values in the Countries ISO dataset is: 0.0 %
There are 0 full duplicates rows
There are 0 empty columns
------------------------------------------------------------------------

Number of values/records per columns in Countries ISO dataset
Name    249
Code    249
dtype: int64


In [16]:
# Filter "EdStatsCountry.csv" based on the list of countries as given in ISO 3166-1
df_country = country[country["2-alpha code"].isin(countries_iso["Code"])]
df_initial_analysis(df_country, "df_country")

The df_country dataset shape is: 213 rows & 32 columns
Total of NaN values in the df_country dataset is: 1687
Percentage of NaN values in the df_country dataset is: 24.75 %
There are 0 full duplicates rows
There are 1 empty columns
   The empty columns are: ['Unnamed: 31']
------------------------------------------------------------------------

Number of values/records per columns in df_country dataset
Country Code                                         213
Table Name                                           213
Long Name                                            213
2-alpha code                                         213
Short Name                                           213
WB-2 code                                            213
Currency Unit                                        212
System of National Accounts                          212
Region                                               211
Income Group                                         211
Latest population censu

In [17]:
df_country.to_csv("filtro.csv", encoding='utf-8')